In [1]:
import requests
import json
from datetime import datetime
from typing import Optional, List, Dict, Any, Generator


from vexa import VexaAPI
vexa = VexaAPI()
await vexa.get_user_info()

BASE_URL = "http://localhost:8765"
headers = {"Authorization": f"Bearer {vexa.token}", "Content-Type": "application/json"}

def submit_token(token: str) -> Dict:
    return requests.post(f"{BASE_URL}/submit_token", json={"token": token}).json()

def chat(query: str, thread_id: Optional[str] = None, model: str = "gpt-4o-mini", temperature: float = 0.7) -> Dict:
    response = requests.post(f"{BASE_URL}/chat", headers=headers, json={
        "query": query, "thread_id": thread_id, "model": model, "temperature": temperature
    }, stream=True)
    
    final_response = None
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8').replace('data: ', ''))
                if data.get('type') == 'stream':
                    print(data.get('content', ''), end='', flush=True)
                elif data.get('type') == 'done':
                    break
                else:
                    final_response = data
            except json.JSONDecodeError:
                continue
    
    return final_response or {"error": "No response received"}
def global_search(query: str, limit: int = 200, min_score: float = 0.4) -> Dict:
    return requests.post(f"{BASE_URL}/search/global", headers=headers, 
        json={"query": query, "limit": limit, "min_score": min_score}).json()

def search_transcripts(query: str, meeting_ids: Optional[List[str]] = None, min_score: float = 0.8) -> Dict:
    return requests.post(f"{BASE_URL}/search/transcripts", headers=headers,
        json={"query": query, "meeting_ids": meeting_ids, "min_score": min_score}).json()

def get_meetings(offset: int = 0, limit: int = 50) -> Dict:
    return requests.get(f"{BASE_URL}/meetings/all", headers=headers, 
        params={"offset": offset, "limit": limit}).json()

def get_meeting_details(meeting_id: str) -> Dict:
    return requests.get(f"{BASE_URL}/meeting/{meeting_id}/details", headers=headers).json()

def create_share_link(target_email: str, access_level: str = "READ", 
                     expiration_hours: int = 24, include_existing: bool = True) -> Dict:
    return requests.post(f"{BASE_URL}/share-links", headers=headers, json={
        "access_level": access_level, "target_email": target_email,
        "expiration_hours": expiration_hours, "include_existing_meetings": include_existing
    }).json()

def accept_share_link(token: str, accepting_email: str) -> Dict:
    return requests.post(f"{BASE_URL}/share-links/accept", headers=headers,
        json={"token": token, "accepting_email": accepting_email}).json()

def start_indexing(num_meetings: int = 200) -> Dict:
    return requests.post(f"{BASE_URL}/start_indexing", headers=headers,
        json={"num_meetings": num_meetings}).json()

def get_indexing_status() -> Dict:
    return requests.get(f"{BASE_URL}/indexing_status", headers=headers).json()

# Add these functions to the initialization cell with other API functions
def get_threads(meeting_id: Optional[str] = None) -> Dict:
    """Get threads for global or meeting-specific context"""
    if meeting_id:
        return requests.get(f"{BASE_URL}/threads/{meeting_id}", headers=headers).json()
    return requests.get(f"{BASE_URL}/threads", headers=headers).json()

def delete_thread(thread_id: str) -> Dict:
    return requests.delete(f"{BASE_URL}/thread/{thread_id}", headers=headers).json()

def chat_meeting(
    query: str, 
    meeting_ids: List[str], 
    thread_id: Optional[str] = None, 
    model: str = "gpt-4o-mini", 
    temperature: float = 0.7
) -> Dict:
    response = requests.post(
        f"{BASE_URL}/chat/meeting", 
        headers=headers, 
        json={
            "query": query,
            "meeting_ids": meeting_ids,
            "thread_id": thread_id,
            "model": model,
            "temperature": temperature
        },
        stream=True
    )
    
    final_response = None
    for line in response.iter_lines():
        if line:
            try:
                data = json.loads(line.decode('utf-8').replace('data: ', ''))
                if data.get('type') == 'stream':
                    print(data.get('content', ''), end='', flush=True)
                elif data.get('type') == 'done':
                    break
                else:
                    final_response = data
            except json.JSONDecodeError:
                continue
    
    return final_response or {"error": "No response received"}

def submit_token(token: str) -> Dict:
    response = requests.post(f"{BASE_URL}/submit_token", json={"token": token})
    result = response.json()
    print(f"User ID: {result.get('user_id')}")
    print(f"User Name: {result.get('user_name')}")
    print(f"Image URL: {result.get('image')}")
    return result

# Example usage:
# r = submit_token(vexa.token)

Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


In [4]:
r = submit_token('48011411848945be98b853b515dd4400')


User ID: 058980af-dfa0-4dc1-b9c0-adf6c24e4994
User Name: Евгения Грудцева
Image URL: https://lh3.googleusercontent.com/a/ACg8ocIO7bXqB1NMoidwvpbMTTyBYZiQWPk33qXjJuI9k1b3m9pxK_8=s96-c


In [6]:
from psql_models import Thread as ThreadModel
from psql_helpers import read_table_async

In [8]:
await read_table_async(ThreadModel)


,thread_id,user_id,meeting_id,thread_name,messages,timestamp
0,8c9fa1ee-c12d-4108-9681-090b17aae523,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,Summarize this meeting,"[{""role"": ""user"", ""content"": ""Summarize this m...",2024-11-06 09:55:35.422041+00:00
1,d9188b0e-e2fe-40eb-be64-3d4618f35003,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,aa87b543-f5c9-426c-a6ed-77b26f93e0da,Summarize this meeting,"[{""role"": ""user"", ""content"": ""Summarize this m...",2024-11-06 10:00:19.378043+00:00
2,cf75f12e-5205-434a-abad-6e9540ededf1,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,aa87b543-f5c9-426c-a6ed-77b26f93e0da,50 char on the meeting,"[{""role"": ""user"", ""content"": ""50 char on the m...",2024-11-06 10:05:49.794543+00:00
3,876cb571-a014-40e4-a1b8-b76c7250b068,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,ilhya,"[{""role"": ""user"", ""content"": ""ilhya"", ""service...",2024-11-06 10:11:50.570529+00:00
4,6f48ca58-ec7a-4f82-aa88-3380b16e11bd,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,None,юридической практике,"[{""role"": ""user"", ""content"": ""\u044e\u0440\u04...",2024-11-06 10:16:19.776499+00:00
5,99e6f76c-ed68-4202-bc29-c20d4ec2f8dd,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,ab27bb1f-93d5-4500-9865-6f5de4d433ea,test,"[{""role"": ""user"", ""content"": ""test"", ""service_...",2024-11-06 10:30:37.275780+00:00
6,8305bf02-2ff9-40de-ad12-a7c120406fec,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,ab27bb1f-93d5-4500-9865-6f5de4d433ea,test,"[{""role"": ""user"", ""content"": ""test"", ""service_...",2024-11-06 10:30:48.564789+00:00
7,f19cb7c1-3ef6-4399-a259-77078c75306c,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,ab27bb1f-93d5-4500-9865-6f5de4d433ea,test,"[{""role"": ""user"", ""content"": ""test"", ""service_...",2024-11-06 10:32:13.052553+00:00
8,f644b240-45f7-42a6-a16a-f8c686f11ab9,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,ab27bb1f-93d5-4500-9865-6f5de4d433ea,о чем встреча?,"[{""role"": ""user"", ""content"": ""\u043e \u0447\u0...",2024-11-06 10:32:39.037313+00:00
9,f61a83e7-691b-4828-b043-5871ef8673f5,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,ab27bb1f-93d5-4500-9865-6f5de4d433ea,test,"[{""role"": ""user"", ""content"": ""test"", ""service_...",2024-11-06 10:44:47.279472+00:00


In [3]:
r = chat_meeting('о чем встреча?', ['ab27bb1f-93d5-4500-9865-6f5de4d433ea'])

Встреча обсуждала переход на новый dashboard и интеграцию с маркетингом. Основные темы включали необходимость наличия нормального локального счета, поиск summary и индекса, отладку Threads, создание контекстного summary из глобального контекста, а также интеграцию с email-маркетингом и календарями. Участники также говорили о необходимости выпуска хотя бы одного поста по маркетингу и о возможности переноса локального серчборда, а также о важности использования более широкого контекста для векторного поиска.

In [4]:
r

{'thread_id': 'a31174fd-475b-40de-855c-3e0fa7d5a9ae',
 'service_content': {'context_source': 'meeting',
  'meeting_count': 1,
  'meeting_id': 'ab27bb1f-93d5-4500-9865-6f5de4d433ea'}}

In [2]:
get_threads('842afeb6-ab24-4a2c-87f2-49a7a1456e12')

[]

In [2]:
r =chat('dmitry')

Dmitriy Grankin has been actively involved in various discussions and projects, focusing on the development of frontend systems, API integration, and knowledge management solutions. Below are key highlights of his contributions:

### Key Contributions

- **Frontend Development**: Dmitriy is developing the frontend for a project that interacts with an API, addressing its functionality and potential issues[1].
- **API Development**: He completed the API development for the project, which is now ready for integration with the frontend[1].
- **Knowledge Base Creation**: Dmitriy is finalizing a knowledge base that compiles information from Google Meet calls, making it accessible for future reference[2].
- **Search Capabilities**: He is implementing search capabilities to enhance access to meeting details, allowing users to query specific information effectively[2].
- **Meeting Summaries**: Dmitriy can generate summaries of marketing efforts and team activities from the knowledge base, facil

In [31]:
r

{'thread_id': '269f19f0-ef3f-46aa-9adb-d28d4910bfdd',
 'service_content': {'context_source': 'meeting',
  'meeting_count': 1,
  'meeting_id': 'ab27bb1f-93d5-4500-9865-6f5de4d433ea'}}